## CNN logistic


Only photos with max 960 in axis X and Y

MSE 



In [1]:
# !pip install keras
# !pip install tensorflow-addons

In [ ]:
import sys
sys.path.append('../src')

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image

from tensorflow.keras import Sequential, Input, layers
from keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow_addons.metrics import FBetaScore

# from tensorflow.keras.metrics import Recall, Precision
# from tensorflow_addons.metrics import F1Score

from utils.data import get_best_slice, select_tomo_ids, get_csv_from_bq
from utils.render_motor_position import get_motor_coordinates, get_slice_file_path


/Users/wm/.pyenv/versions/3.10.6/envs/byu/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/Users/wm/.pyenv/versions/3.10.6/envs/byu/lib/python3.10/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.13.0 and strictly below 2.16.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter 

In [3]:
path_train_csv= '../data/csv_raw/train_labels.csv'
# /Users/wm/code/mc-will/bacterial_motor/BYU_Locating-Bacterial_Flagellar_Motors_2025/data/csv_raw/train_labels.csv
path_image= '../data/pictures_raw/train'

df = pd.read_csv(path_train_csv).copy()
df

,Unnamed: 0,row_id,tomo_id,Motor_axis_0,Motor_axis_1,Motor_axis_2,Array_shape_axis_0,Array_shape_axis_1,Array_shape_axis_2,Voxel_spacing,Number_of_motors
0,0,16,tomo_049310,-1.0,-1.0,-1.0,500,924,956,19.7,0
1,1,30,tomo_098751,-1.0,-1.0,-1.0,500,924,956,16.1,0
2,2,54,tomo_136c8d,-1.0,-1.0,-1.0,500,924,956,19.7,0
3,3,59,tomo_146de2,-1.0,-1.0,-1.0,500,924,956,16.1,0
4,4,84,tomo_1dc5f9,-1.0,-1.0,-1.0,500,924,956,19.7,0
...,...,...,...,...,...,...,...,...,...,...,...
732,732,99,tomo_226cd8,366.0,693.0,269.0,500,928,960,13.1,10
733,733,100,tomo_226cd8,386.0,693.0,319.0,500,928,960,13.1,10
734,734,101,tomo_226cd8,388.0,671.0,433.0,500,928,960,13.1,10
735,735,102,tomo_226cd8,411.0,318.0,740.0,500,928,960,13.1,10


In [5]:
tomogram_id= select_tomo_ids(df, );


In [6]:
# df['tomo_id'].reset_index(drop = True)==tomogram_id.reset_index(drop = True)
def get_best_slice(df, tomogram_id):
    '''
    Find the coordinates of the motor (only work with 1 motor tomogram), the use the z to get the picture_path where
    the motor is located, i.e. the 'best slice'

            Parameters:
                df (pd.Dataframe): the dataset to find coordinates
                tomogram_id (string): the tomogram id

            Returns:
                best_slice_path (string): the path of the picture of the 'best slice' (where the motor is located)

    '''
    x, y, z = get_motor_coordinates(df, tomogram_id)
    if z <0:
        best_slice = 150
        best_slice_path = get_slice_file_path(tomogram_id, int(best_slice))
    else:
        best_slice_path = get_slice_file_path(tomogram_id, int(z))
    return best_slice_path

In [7]:
#  /Users/wm/code/mc-will/bacterial_motor/BYU_Locating-Bacterial_Flagellar_Motors_2025/data/pictures_raw/train/tomo_0a8f05/slice_0000.jpg
best_slice =[get_best_slice(df,tomo) for tomo in tomogram_id]


In [8]:

def load_slices(best_slice):
    slices = []
    for i in best_slice:
        relative_path = i.lstrip('./')
        img_path = Path('..') / relative_path


        img = load_img(img_path, color_mode='grayscale')
        img_array = img_to_array(img) / 255.0
        slices.append(img_array)

    return np.array(slices)


In [9]:
# """
# Function look the foto 72 in each 'tomo'
# * Img size is too heavy at 960*960 pixel, so se test on 256*256

# """
# def load_from_each_tomo(parent_folder,
#                         img_size=(960, 960),
#                         # img_size=(256,256),
#                         # img_size=(128, 128),
#                         ):

#     slices = []
#     # select the column of 'tomo_id' with only 1/0 Motors
#     tomo_ids = df['tomo_id']
#     # print (tomo_ids)

#     # Loop to look each foto per Tomo_id
#     for id in tomo_ids:
#         # make the path btw parent_path and tomo_id
#         tomo_path = os.path.join(parent_folder, id)
#         # print(tomo_path)

#         # Check if it is a real directory
#         if os.path.isdir(tomo_path):
#             # make a liste of all the photo per tomo
#             all_slices = sorted([f for f in os.listdir(tomo_path)])
#             # print(' all slices:',len(all_slices), type(all_slices), all_slices)

#             number = 72 ### mean where the best photo is

#             slice_path =os.path.join(tomo_path, all_slices[number])
#             # load the img from slice_path with same size , in 1 chanel = W/B
#             img = load_img(slice_path,
#                            target_size=img_size,
#                            color_mode='grayscale',
#                            )
#             # Normalization btw 0/1 divide by 255
#             img_array = img_to_array(img) / 255

#             # append to the slice list
#             slices.append(img_array)

#     # return un array to be used
#     return np.array(slices)


In [10]:
# selection of y= 1 or 0 Motors

X =load_slices(best_slice);
y= df['Number_of_motors']

/var/folders/nr/0rc2wzzd26zd1tptqppt9kjr0000gn/T/ipykernel_18218/1305404653.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(slices)


In [11]:

target_size = (928, 928)
slices_resized = []

for i, img in enumerate(X):

    pil_img = Image.fromarray((img * 255).astype('uint8').squeeze())  # out one canal if it is 1
    pil_img = pil_img.resize(target_size)
    img_array = img_to_array(pil_img) / 255.0
    if img_array.ndim == 2:  # si vuelve a perder canal
        img_array = np.expand_dims(img_array, axis=-1)
    slices_resized.append(img_array)

X = np.stack(slices_resized)


In [12]:

def init_model_cnnlog(X):
    model = Sequential()
    model.add(Input(shape= X.shape[1:]))

    model.add(layers.Conv2D(32, (3, 3),
                            padding='same',
                            activation="relu"))
    model.add(layers.MaxPool2D(pool_size=(2, 2)))
    #
    model.add(layers.Conv2D(64, (3, 3),
                            padding='same',
                            activation="relu"))
    model.add(layers.MaxPool2D(pool_size=(2, 2)))

    model.add(layers.Conv2D(128, (3, 3),
                            padding='same',
                            activation="relu"))
    model.add(layers.BatchNormalization())

    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.3))


    model.add(layers.Dense(1, activation='sigmoid'))


    ########

    model.compile(loss= 'binary_crossentropy',
                  optimizer = 'adam',
                #   metrics=['mse', 'F1Score'],
                  metrics=['rsme',
                           FBetaScore(num_classes=1,
                                      average='micro',
                                      beta=2.0),
                        #    Recall(),
                        #    'precision',
                        #    'F1Score'
                           ],)

    return model


In [13]:
model = init_model_cnnlog(X)

In [ ]:
def model_train (model, X_train_processed, y_train, batch_size=8 ,epochs=10):
    verbose=1
    validation_split=0.1
    es = EarlyStopping(patience=5,
                       restore_best_weights=True)

    history = model.fit(X_train_processed,
                        y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_split=validation_split,
                        callbacks=[es],
                        verbose=verbose)
    return history
history = model_train(model, X,y)
history

Epoch 1/10


2025-06-04 17:50:53.893406: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


10/47 [=====>........................] - ETA: 1:31:22 - loss: 42.9112 - accuracy: 0.6750 - fbeta_score: 0.4124

In [1]:
history.history


NameError: name 'history' is not defined

In [ ]:
# plt.plot(history.history['mse'],c='b');
# plt.plot(history.history['loss'], c='r');
# plt.plot(history.history['val_loss'],c='g');
plt.plot(history.history['F1Score'], c='r');
# plt.plot(history.history['accuracy'],c='black');
# plt.plot(history.history['val_accuracy'],'b');
plt.plot(history.history['recall'], c= 'r');
# plt.plot(history.history['val_recall']);
plt.plot(history.history['precision'], c='g');
# plt.plot(history.history['val_precision'], c='b');


In [ ]:
model.summary()

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [5]:
datagen